In [1]:
'''Import necessary libraries'''
import requests
import pandas as pd
from bs4 import BeautifulSoup as bs
import mechanicalsoup as ms
import re
import polling2, time
import os, glob
import math

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

from PyPDF2 import PdfReader as pypdf
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import pickle

In [ ]:
'''Retrieve webpage info for landing page for Wilmington, NC crime reports'''
url = 'http://p2c.wilmingtonnc.gov/Summary_Disclaimer.aspx'

'''Open browser with URL'''
browser = ms.StatefulBrowser()

#get page
browser.open(url)
#find all forms on page Note....browser.page in a bs4 object
browser.select_form('form[action="./Summary_Disclaimer.aspx"]')

In [ ]:
'''Submit initial page'''
response = browser.submit_selected()
print(response.text)

In [ ]:
'''Select form from main page'''
browser.select_form()
browser.form.print_summary()

In [ ]:
browser.page.find_all('input')[3]['unchecked'] = ''
browser["MasterPage$mainContent$txtDateFrom2"] = '5/1/2022'
browser["MasterPage$mainContent$CGeoCityDDL12"] = 'WILMINGTON'
browser["MasterPage$mainContent$ddlRange2"] = '0.378788'
browser["MasterPage$mainContent$txtStreetNo2"] = '219'
browser["MasterPage$mainContent$txtStreetName2"] = 'S 6th ST'

In [ ]:
browser.launch_browser()

In [ ]:
response = browser.submit_selected()

In [ ]:
response.soup

In [ ]:
browser.close()

## Same thing as above with Selenium

In [2]:
'''Declare driver and get initial webpage'''
url = 'http://p2c.wilmingtonnc.gov/Summary_Disclaimer.aspx'

options = Options()
options.headless = True
options.add_argument("--window-size=1920,1200")
download_directory = r'C:\Users\scott\Data Projects\Crime Stats\pdf_files'
p = {'download.default_directory': download_directory}
options.add_experimental_option("prefs", p)

driver = webdriver.Chrome(options=options, service=Service(ChromeDriverManager().install()))
driver.get(url)
print(driver.page_source)

<html><head><meta http-equiv="Page-Enter" content="Alpha(opacity=100)"><meta http-equiv="content-type" content="text/html; charset=utf-8"><link id="activeStylesheetLink" href="css/active.css" rel="Stylesheet"><link id="commonStylesheet" href="css/p2cBase.css" rel="Stylesheet"><link rel="Shortcut Icon" href="./images/favicon.ico" type="image/x-icon"><script src="js/jquery-1.8.3.min.js" type="text/javascript"></script><script src="jqui/1.8/jquery-ui-1.8.custom.min.js" type="text/javascript"></script><link href="jqui/1.8/black-tie/jquery-ui.css" rel="Stylesheet" class="ui-theme"><link href="jqui/1.8/black-tie/theme.css" rel="Stylesheet" class="ui-theme">
    
<link id="jquiOverride" href="css/jqui-override.css" rel="Stylesheet">
    <script language="javascript" type="text/javascript">
        // DO NOT REMOVE
        // this function is used by the form builder
        // to control tab index issues
        function getASPElm(nm, tag) {
            if ($get(nm)) { return $get(nm); }
    

In [3]:
'''Click submit on initial page'''
driver.find_element(By.NAME, "ctl00$MasterPage$mainContent$CenterColumnContent$btnContinue").click()

In [4]:
print(driver.page_source)

<html><head><meta http-equiv="Page-Enter" content="Alpha(opacity=100)"><meta http-equiv="content-type" content="text/html; charset=utf-8"><link id="activeStylesheetLink" href="css/active.css" rel="Stylesheet"><link id="commonStylesheet" href="css/p2cBase.css" rel="Stylesheet"><link rel="Shortcut Icon" href="./images/favicon.ico" type="image/x-icon"><script src="js/jquery-1.8.3.min.js" type="text/javascript"></script><script src="jqui/1.8/jquery-ui-1.8.custom.min.js" type="text/javascript"></script><link href="jqui/1.8/black-tie/jquery-ui.css" rel="Stylesheet" class="ui-theme"><link href="jqui/1.8/black-tie/theme.css" rel="Stylesheet" class="ui-theme">
    <script type="text/javascript">
        //window.onload = function() { window.name = "p2c_summary" };
        window.name = "p2c_summary";
        function ModifyWaitMessage(link) {
            if (link.innerHTML.toUpperCase().indexOf('CART') > -1) {
                _WaitControlMessageID.innerHTML = '<font size=4>Please Wait</font><br

In [5]:
'''Complete form fields'''
driver.find_element(By.NAME, "MasterPage$mainContent$txtDateFrom2").clear()
driver.find_element(By.NAME, "MasterPage$mainContent$txtDateFrom2").send_keys('5/25/2022')

driver.find_element(By.ID, "mainContent_chkTA2").click()

try: 
    driver.find_element(By.NAME, "MasterPage$mainContent$CGeoCityDDL12").clear()
except:
    pass
driver.find_element(By.NAME, "MasterPage$mainContent$CGeoCityDDL12").send_keys('WILMINGTON')

try:
    driver.find_element(By.NAME, "MasterPage$mainContent$ddlRange2").clear()
except:
    pass
driver.find_element(By.NAME, "MasterPage$mainContent$ddlRange2").send_keys('1000')

try:
    driver.find_element(By.NAME, "MasterPage$mainContent$txtStreetNo2").clear()
except:
    pass
driver.find_element(By.NAME, "MasterPage$mainContent$txtStreetNo2").send_keys('219')

try:
    driver.find_element(By.NAME, "MasterPage$mainContent$txtStreetName2").clear()
except:
    pass
driver.find_element(By.NAME, "MasterPage$mainContent$txtStreetName2").send_keys('S 6TH ST')

In [6]:
'''Submit form via javascript'''
submit =  driver.find_element(By.ID, "mainContent_cmdSubmit2")
driver.execute_script("arguments[0].click()", submit)

In [7]:
'''Take screenshot of the browser'''
try:
    driver.save_screenshot('screenshot.png')
except:
    pass

In [ ]:
'''data_rows = driver.find_elements(By.XPATH, ('//table[@class="DataGridText"]//tr[@class = "EventSearchGridRow"]//td[position() = 4]'))

for i in data_rows:
    if re.search(r"Primary Offense:(.*)", i.text):
        offense = re.search(r"Primary Offense:(.*)", i.text)
        print(offense.group(1))'''

In [8]:
'''Save each PDF file to pdf directory'''
#Get number of pages in results table and place it into a list - this is not the total, but the number of pages currently listed
number_current_pages = driver.find_elements(By.XPATH, ('//table[@class="DataGridText"]//tr[@align = "center"]//td//table'))[0].text.split(' ')

#Get overall total number of pages
number_total_pages = driver.find_elements(By.XPATH, ('//span[@id="mainContent_lblRecordCount"]'))[0].text
number_total_pages = re.search(r"\d+", number_total_pages)
number_total_pages = (math.ceil((int(number_total_pages.group(0))/9)))
#print(number_total_pages)

#Create empty list into which to place items as they are extracted
crime_list = []
injury_list = []
relationship_list = []
report_type_list = []
block_list = []
address_list = []
date_list = []

#Cycle through each page in list
current_page = 1
cum_page_count = 0

for page in range(len(number_current_pages)):
    #Extract report type, date and address from results table
    report_type = driver.find_elements(By.XPATH, ('//table[@class="DataGridText"]//tr[@class = "EventSearchGridRow"]//td[position() = 3]'))   

    #Cycle through each document on the page
    for index in range(len(report_type)):
        found_crimes = 0
        found_injuries = 0
        found_relationships = 0
        found_addresses = 0
        
        #Create pdfReader objects out of each file
        if cum_page_count % 9 == 0 & cum_page_count != 0:
            pdf_file_link = driver.find_element(By.ID, f"mainContent_gvSummary_lbGetReport_{index - 1}")
            #print("-1:",index)
        else:
            pdf_file_link = driver.find_element(By.ID, f"mainContent_gvSummary_lbGetReport_{index}")
            #print("-1:",index)
            
        print(f"mainContent_gvSummary_lbGetReport_{index}")
        pdf_file_link.click()
        time.sleep(5)
        os.rename(download_directory+r"\PoliceReport.pdf", download_directory+fr"\PoliceReport{index}.pdf")
        time.sleep(3)
        
        # Within each page, cycle through each line of the data table and extract information
        report_type = driver.find_elements(By.XPATH, ('//table[@class="DataGridText"]//tr[@class = "EventSearchGridRow"]//td[position() = 3]'))
        dates = driver.find_elements(By.XPATH, ('//table[@class="DataGridText"]//tr[@class = "EventSearchGridRow"]//td[position() = 2]'))
        blocks = driver.find_elements(By.XPATH, ('//table[@class="DataGridText"]//tr[@class = "EventSearchGridRow"]//td[position() = 5]'))

        if report_type[index].text == "Incident":
            report_type_list.append(report_type[index].text)
            #print(report_type[index].text)
            date_list.append(dates[index].text)
            #print(dates[index].text)
            block_list.append(blocks[index].text)
            #print(blocks[index].text)
                  
            with open(fr"pdf_files/PoliceReport{index}.pdf", 'rb') as f:
                pdfReader = pypdf(f, "rb")


                #create list of text from page
                page_object = pdfReader.getPage(0)
                page_text = page_object.extractText()
                page = page_text.split("\n")
                address_list_temp = []
                for i in page:
                    found_crimes_loop = 0
                    if page.index(i) == 34:
                        crime_list.append(i)
                        found_crimes_loop += 1
                        found_crimes = found_crimes_loop
                        #print(found_crimes)

                    injury = re.search(r"\d+/\d+/\d{4}(.*)com", i, re.IGNORECASE)
                    found_injuries_loop = 0
                    if injury == '':
                        #print('blank injury')
                        injury_list.append('None')
                        found_injuries_loop += 1
                        found_injuries = found_injuries_loop
                    elif injury is not None:
                        #print('none injury')
                        injury_list.append(injury.group(1))
                        found_injuries_loop += 1
                        found_injuries = found_injuries_loop
                                            

                    relationship = re.search(r"(\d[A-Z]{2})$", i)
                    found_relationships_loop = 0
                    if relationship is not None:
                        relationship_list.append(relationship.group(1))
                        found_relationships_loop += 1
                        found_relationships = found_relationships_loop
                        print(relationship.group(1))
                        
                    address = re.search(r"\d+ (.*) (Wilmington|WM)", i)
                    found_addresses_loop = 0
                
                    if address is not None:
                        print(address.group(0).split(',')[0])
                        address_list_temp.append(address.group(0).split(',')[0][5:])
                        print(address_list_temp)
                        found_addresses_loop += 1
                        found_addresses = found_addresses_loop
                if len(address_list_temp) > 0:
                    address_list.append(address_list_temp[0])
                print(address_list)

                #If any item was not present, append list with None
                if found_injuries == 0:
                    injury_list.append('None')
                if found_crimes == 0:
                    crime_list.append('None')
                #print('found_crimes_end', found_crimes)
                if found_relationships == 0:
                    relationship_list.append('None')
                if found_addresses == 0:
                    address_list.append('None')
                print('crime_list', len(crime_list), crime_list)
                print('injury list', len(injury_list), injury_list)
                print('relationship_list', len(relationship_list), relationship_list)
                print('address_list', len(address_list), address_list)
                print('report_type_list', len(report_type_list), report_type_list)
                print('block_list', len(block_list), block_list)
                print('address_list', len(address_list), address_list)
                print('date_list', len(date_list), date_list)

        cum_page_count += 1
    current_page += 1
    print(current_page)
    new_page = driver.find_element(By.XPATH, f'//a[@href = "javascript:__doPostBack(\'MasterPage$mainContent$gvSummary\',\'Page${current_page}\')"]')
    print(new_page.text)
    driver.execute_script("arguments[0].click()", new_page)
    driver.save_screenshot('screenshot.png')
    number_current_pages = driver.find_elements(By.XPATH, ('//table[@class="DataGridText"]//tr[@align = "center"]//td//table'))[0].text.split(' ')
    #Remove saved files from previous page iteration
    files = os.listdir(download_directory)
    filtered_files = [file for file in files if file.endswith(".pdf")]
    for file in filtered_files:
        path_to_file = os.path.join(download_directory, file)
        os.remove(path_to_file)
print('crime_list', crime_list)
print('injury list', injury_list)
print('relationship_list', relationship_list)
print('address_list',address_list)
print('report_type_list', report_type_list)
print('block_list', block_list)
print('address_list', address_list)
print('date_list', date_list)

mainContent_gvSummary_lbGetReport_0
2022 399 S 8th St
['399 S 8th St']
['399 S 8th St']
crime_list 1 ['Drug/narcotic Violations']
injury list 1 ['']
relationship_list 1 ['None']
address_list 1 ['399 S 8th St']
report_type_list 1 ['Incident']
block_list 1 ['300-BLK S 8TH ST']
address_list 1 ['399 S 8th St']
date_list 1 ['05/25/2022 00:33']
mainContent_gvSummary_lbGetReport_1
mainContent_gvSummary_lbGetReport_2
2022 702 Ann St
['702 Ann St']
2523 CONFEDERATE DR 
['702 Ann St', 'CONFEDERATE DR ']
['399 S 8th St', '702 Ann St']
crime_list 2 ['Drug/narcotic Violations', 'Financial Transaction Card Fraud']
injury list 2 ['', '']
relationship_list 2 ['None', 'None']
address_list 2 ['399 S 8th St', '702 Ann St']
report_type_list 2 ['Incident', 'Incident']
block_list 2 ['300-BLK S 8TH ST', '700-BLK ANN ST']
address_list 2 ['399 S 8th St', '702 Ann St']
date_list 2 ['05/25/2022 00:33', '05/26/2022 22:00']
mainContent_gvSummary_lbGetReport_3
2022 404 S 5th Av
['404 S 5th Av']
417 Windmere Rd  Wil

mainContent_gvSummary_lbGetReport_5
2022 411 S 7th St
['411 S 7th St']
411 S 7th St  Wilmington
['411 S 7th St', ' 7th St  Wilmington']
411 S 7TH ST 
['411 S 7th St', ' 7th St  Wilmington', ' 7TH ST ']
['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St', '510 Dock St', '411 S 7th St']
crime_list 10 ['Drug/narcotic Violations', 'Financial Transaction Card Fraud', 'Missing Persons', 'Resisting Arrest/hinder & Delay', 'Mv Theft', 'B&e Auto Larceny From Auto', 'Miscellaneous', 'Mv Theft', 'Possession Of  Stolen Vehicles', 'B&e Felony']
injury list 10 ['', '', '', '', '', '', '', '', '', '']
relationship_list 10 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']
address_list 10 ['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St', '510 Dock St', '411 S 7th St']
report_type_list 10 ['Incident', 'Incident', 'Incident', 'Incident

2022 224 S 4th St
['224 S 4th St']
224 S 4TH ST 
['224 S 4th St', ' 4TH ST ']
['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St', '510 Dock St', '411 S 7th St', '332 S 8th St', '319 S 5th Av', '316 Ann St', '108 S 5th Av', '224 S 4th St']
crime_list 15 ['Drug/narcotic Violations', 'Financial Transaction Card Fraud', 'Missing Persons', 'Resisting Arrest/hinder & Delay', 'Mv Theft', 'B&e Auto Larceny From Auto', 'Miscellaneous', 'Mv Theft', 'Possession Of  Stolen Vehicles', 'B&e Felony', 'Larceny - All Other', 'Dwi - Alcohol', 'Larceny - All Other', 'Larceny - All Other', 'Larceny - All Other']
injury list 15 ['', '', '', '', '', '', '', '', '', '', '', '', '', '', '']
relationship_list 15 ['None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None', 'None']
address_list 15 ['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608

2022 512 Nun St
['512 Nun St']
315 Dawson St  Wilmington
['512 Nun St', 'awson St  Wilmington']
6204 Old Military Rd  Wilmington
['512 Nun St', 'awson St  Wilmington', 'Old Military Rd  Wilmington']
1311 41ST ST 
['512 Nun St', 'awson St  Wilmington', 'Old Military Rd  Wilmington', '41ST ST ']
['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St', '510 Dock St', '411 S 7th St', '332 S 8th St', '319 S 5th Av', '316 Ann St', '108 S 5th Av', '224 S 4th St', '299 S 6th St', '117 S 7th St', '712 Dock St', '512 Nun St']
crime_list 19 ['Drug/narcotic Violations', 'Financial Transaction Card Fraud', 'Missing Persons', 'Resisting Arrest/hinder & Delay', 'Mv Theft', 'B&e Auto Larceny From Auto', 'Miscellaneous', 'Mv Theft', 'Possession Of  Stolen Vehicles', 'B&e Felony', 'Larceny - All Other', 'Dwi - Alcohol', 'Larceny - All Other', 'Larceny - All Other', 'Larceny - All Other', 'Overdose', 'Larceny - From Auto', 'Hit & Run-misd F

2022 715 Ann St
['715 Ann St']
715 Ann St  Wilmington
['715 Ann St', 'nn St  Wilmington']
715 ANN ST 
['715 Ann St', 'nn St  Wilmington', 'NN ST ']
['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St', '510 Dock St', '411 S 7th St', '332 S 8th St', '319 S 5th Av', '316 Ann St', '108 S 5th Av', '224 S 4th St', '299 S 6th St', '117 S 7th St', '712 Dock St', '512 Nun St', '212 S 7th St Apt. 4', '705 Orange St', '715 Ann St']
crime_list 22 ['Drug/narcotic Violations', 'Financial Transaction Card Fraud', 'Missing Persons', 'Resisting Arrest/hinder & Delay', 'Mv Theft', 'B&e Auto Larceny From Auto', 'Miscellaneous', 'Mv Theft', 'Possession Of  Stolen Vehicles', 'B&e Felony', 'Larceny - All Other', 'Dwi - Alcohol', 'Larceny - All Other', 'Larceny - All Other', 'Larceny - All Other', 'Overdose', 'Larceny - From Auto', 'Hit & Run-misd Fail To Stop', 'Fire Of Unk Origin', 'Larceny - From Auto', 'Larceny - From Auto', 'Death Inve

2022 200 S 7th St
['200 S 7th St']
615 Bess St  Wilmington
['200 S 7th St', 'ess St  Wilmington']
['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St', '510 Dock St', '411 S 7th St', '332 S 8th St', '319 S 5th Av', '316 Ann St', '108 S 5th Av', '224 S 4th St', '299 S 6th St', '117 S 7th St', '712 Dock St', '512 Nun St', '212 S 7th St Apt. 4', '705 Orange St', '715 Ann St', '516 Dock St', '700 Ann St', '200 S 7th St']
crime_list 25 ['Drug/narcotic Violations', 'Financial Transaction Card Fraud', 'Missing Persons', 'Resisting Arrest/hinder & Delay', 'Mv Theft', 'B&e Auto Larceny From Auto', 'Miscellaneous', 'Mv Theft', 'Possession Of  Stolen Vehicles', 'B&e Felony', 'Larceny - All Other', 'Dwi - Alcohol', 'Larceny - All Other', 'Larceny - All Other', 'Larceny - All Other', 'Overdose', 'Larceny - From Auto', 'Hit & Run-misd Fail To Stop', 'Fire Of Unk Origin', 'Larceny - From Auto', 'Larceny - From Auto', 'Death Investiga

mainContent_gvSummary_lbGetReport_5
mainContent_gvSummary_lbGetReport_6
2022 711 Nun St
['711 Nun St']
['399 S 8th St', '702 Ann St', '404 S 5th Av', '700 Nun St', '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St', '510 Dock St', '411 S 7th St', '332 S 8th St', '319 S 5th Av', '316 Ann St', '108 S 5th Av', '224 S 4th St', '299 S 6th St', '117 S 7th St', '712 Dock St', '512 Nun St', '212 S 7th St Apt. 4', '705 Orange St', '715 Ann St', '516 Dock St', '700 Ann St', '200 S 7th St', '410 S 7th St', '700 Ann St', '711 Nun St']
crime_list 28 ['Drug/narcotic Violations', 'Financial Transaction Card Fraud', 'Missing Persons', 'Resisting Arrest/hinder & Delay', 'Mv Theft', 'B&e Auto Larceny From Auto', 'Miscellaneous', 'Mv Theft', 'Possession Of  Stolen Vehicles', 'B&e Felony', 'Larceny - All Other', 'Dwi - Alcohol', 'Larceny - All Other', 'Larceny - All Other', 'Larceny - All Other', 'Overdose', 'Larceny - From Auto', 'Hit & Run-misd Fail To Stop', 'Fire Of Unk Origin', 'Larceny - Fr

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//a[@href = "javascript:__doPostBack('MasterPage$mainContent$gvSummary','Page$6')"]"}
  (Session info: headless chrome=107.0.5304.123)
Stacktrace:
Backtrace:
	Ordinal0 [0x00A6ACD3+2075859]
	Ordinal0 [0x009FEE61+1633889]
	Ordinal0 [0x008FB7BD+571325]
	Ordinal0 [0x0092AC2F+764975]
	Ordinal0 [0x0092AE1B+765467]
	Ordinal0 [0x0095D0F2+970994]
	Ordinal0 [0x00947364+881508]
	Ordinal0 [0x0095B56A+963946]
	Ordinal0 [0x00947136+880950]
	Ordinal0 [0x0091FEFD+720637]
	Ordinal0 [0x00920F3F+724799]
	GetHandleVerifier [0x00D1EED2+2769538]
	GetHandleVerifier [0x00D10D95+2711877]
	GetHandleVerifier [0x00AFA03A+521194]
	GetHandleVerifier [0x00AF8DA0+516432]
	Ordinal0 [0x00A0682C+1665068]
	Ordinal0 [0x00A0B128+1683752]
	Ordinal0 [0x00A0B215+1683989]
	Ordinal0 [0x00A16484+1729668]
	BaseThreadInitThunk [0x75EA7BA9+25]
	RtlInitializeExceptionChain [0x76F4BB9B+107]
	RtlClearBits [0x76F4BB1F+191]


In [9]:
'''Check for consistency of list lengths'''
complete_list = [crime_list, injury_list, relationship_list, report_type_list, block_list, address_list, date_list]
for list in complete_list:
    print(len(list))

28
28
28
28
28
28
28


In [10]:
'''Create a dataframe from the lists'''
complete_list = [crime_list, injury_list, relationship_list, report_type_list, block_list, address_list, date_list]
df = pd.DataFrame(tuple(zip(crime_list, injury_list, relationship_list, report_type_list, block_list, address_list, date_list)), columns = ['crime', 'injury', 'relationship', 'report_type', 'block', 'address', 'date'])
df.loc[df['crime'].str.contains('larceny', case=False)]

,crime,injury,relationship,report_type,block,address,date
5,B&e Auto Larceny From Auto,,None,Incident,300-BLK S 6TH ST,307 S 6th St,06/02/2022 11:58
10,Larceny - All Other,,None,Incident,300-BLK S 8TH ST,332 S 8th St,07/21/2022 22:00
12,Larceny - All Other,,None,Incident,300-BLK ANN ST,316 Ann St,08/15/2022 09:00
13,Larceny - All Other,,None,Incident,100-BLK S 5TH AV,108 S 5th Av,08/16/2022 02:20
14,Larceny - All Other,,None,Incident,200-BLK S 4TH ST,224 S 4th St,08/19/2022 18:00
16,Larceny - From Auto,,None,Incident,100-BLK S 7TH ST,117 S 7th St,09/06/2022 23:00
19,Larceny - From Auto,,None,Incident,200-BLK S 7TH ST,212 S 7th St Apt. 4,09/24/2022 23:30
20,Larceny - From Auto,,None,Incident,700-BLK ORANGE ST,705 Orange St,09/27/2022 08:30
22,Larceny - All Other,,None,Incident,500-BLK DOCK ST,516 Dock St,10/05/2022 20:15


In [11]:
'''Pickle crime_rate data_frame'''
with open('crime_rates.pkl', 'wb') as f:
    pickle.dump(df, f)

In [12]:
'''Open for_sale pickle file'''
with open('crime_rates.pkl', 'rb') as f:
    df = pickle.load(f)

In [13]:
df

,crime,injury,relationship,report_type,block,address,date
0,Drug/narcotic Violations,,None,Incident,300-BLK S 8TH ST,399 S 8th St,05/25/2022 00:33
1,Financial Transaction Card Fraud,,None,Incident,700-BLK ANN ST,702 Ann St,05/26/2022 22:00
2,Missing Persons,,None,Incident,400-BLK S 5TH AV,404 S 5th Av,05/28/2022 12:00
3,Resisting Arrest/hinder & Delay,,None,Incident,700-BLK NUN ST,700 Nun St,05/30/2022 22:58
4,Mv Theft,,None,Incident,500-BLK DOCK ST,500 Dock St,06/01/2022 15:00
5,B&e Auto Larceny From Auto,,None,Incident,300-BLK S 6TH ST,307 S 6th St,06/02/2022 11:58
6,Miscellaneous,,None,Incident,600-BLK DOCK ST,608 Dock St,06/05/2022 11:14
7,Mv Theft,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00
8,Possession Of Stolen Vehicles,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00
9,B&e Felony,,None,Incident,400-BLK S 7TH ST,411 S 7th St,07/01/2022 22:00


In [14]:
'''Standardize addresses'''
df['address'] = df['address'].str.extract(r'(\d+ .* St).*')
df

,crime,injury,relationship,report_type,block,address,date
0,Drug/narcotic Violations,,None,Incident,300-BLK S 8TH ST,399 S 8th St,05/25/2022 00:33
1,Financial Transaction Card Fraud,,None,Incident,700-BLK ANN ST,702 Ann St,05/26/2022 22:00
2,Missing Persons,,None,Incident,400-BLK S 5TH AV,NaN,05/28/2022 12:00
3,Resisting Arrest/hinder & Delay,,None,Incident,700-BLK NUN ST,700 Nun St,05/30/2022 22:58
4,Mv Theft,,None,Incident,500-BLK DOCK ST,500 Dock St,06/01/2022 15:00
5,B&e Auto Larceny From Auto,,None,Incident,300-BLK S 6TH ST,307 S 6th St,06/02/2022 11:58
6,Miscellaneous,,None,Incident,600-BLK DOCK ST,608 Dock St,06/05/2022 11:14
7,Mv Theft,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00
8,Possession Of Stolen Vehicles,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00
9,B&e Felony,,None,Incident,400-BLK S 7TH ST,411 S 7th St,07/01/2022 22:00


In [15]:
df.loc[df['address'].isnull()]

,crime,injury,relationship,report_type,block,address,date
2,Missing Persons,,None,Incident,400-BLK S 5TH AV,NaN,05/28/2022 12:00
11,Dwi - Alcohol,,None,Incident,300-BLK S 5TH AV,NaN,07/24/2022 05:20
13,Larceny - All Other,,None,Incident,100-BLK S 5TH AV,NaN,08/16/2022 02:20


In [16]:
df.loc[df['address'].isnull(), 'address'] = '200 N 12TH ST'
df

,crime,injury,relationship,report_type,block,address,date
0,Drug/narcotic Violations,,None,Incident,300-BLK S 8TH ST,399 S 8th St,05/25/2022 00:33
1,Financial Transaction Card Fraud,,None,Incident,700-BLK ANN ST,702 Ann St,05/26/2022 22:00
2,Missing Persons,,None,Incident,400-BLK S 5TH AV,200 N 12TH ST,05/28/2022 12:00
3,Resisting Arrest/hinder & Delay,,None,Incident,700-BLK NUN ST,700 Nun St,05/30/2022 22:58
4,Mv Theft,,None,Incident,500-BLK DOCK ST,500 Dock St,06/01/2022 15:00
5,B&e Auto Larceny From Auto,,None,Incident,300-BLK S 6TH ST,307 S 6th St,06/02/2022 11:58
6,Miscellaneous,,None,Incident,600-BLK DOCK ST,608 Dock St,06/05/2022 11:14
7,Mv Theft,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00
8,Possession Of Stolen Vehicles,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00
9,B&e Felony,,None,Incident,400-BLK S 7TH ST,411 S 7th St,07/01/2022 22:00


In [17]:
'''Create full address column'''
df['full_address'] = df['address'] + ', Wilmington, NC 28401'
df

,crime,injury,relationship,report_type,block,address,date,full_address
0,Drug/narcotic Violations,,None,Incident,300-BLK S 8TH ST,399 S 8th St,05/25/2022 00:33,"399 S 8th St, Wilmington, NC 28401"
1,Financial Transaction Card Fraud,,None,Incident,700-BLK ANN ST,702 Ann St,05/26/2022 22:00,"702 Ann St, Wilmington, NC 28401"
2,Missing Persons,,None,Incident,400-BLK S 5TH AV,200 N 12TH ST,05/28/2022 12:00,"200 N 12TH ST, Wilmington, NC 28401"
3,Resisting Arrest/hinder & Delay,,None,Incident,700-BLK NUN ST,700 Nun St,05/30/2022 22:58,"700 Nun St, Wilmington, NC 28401"
4,Mv Theft,,None,Incident,500-BLK DOCK ST,500 Dock St,06/01/2022 15:00,"500 Dock St, Wilmington, NC 28401"
5,B&e Auto Larceny From Auto,,None,Incident,300-BLK S 6TH ST,307 S 6th St,06/02/2022 11:58,"307 S 6th St, Wilmington, NC 28401"
6,Miscellaneous,,None,Incident,600-BLK DOCK ST,608 Dock St,06/05/2022 11:14,"608 Dock St, Wilmington, NC 28401"
7,Mv Theft,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00,"510 Dock St, Wilmington, NC 28401"
8,Possession Of Stolen Vehicles,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00,"510 Dock St, Wilmington, NC 28401"
9,B&e Felony,,None,Incident,400-BLK S 7TH ST,411 S 7th St,07/01/2022 22:00,"411 S 7th St, Wilmington, NC 28401"


In [19]:
'''Obtain longitude and latitude for each address'''

#create locator object
locator = Nominatim(user_agent= "myGeocoder")

#delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

#create location column
df['location'] = df['full_address'].apply(geocode)

#create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)

#split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

In [20]:
df

,crime,injury,relationship,report_type,block,address,date,full_address,location,point,latitude,longitude,altitude
0,Drug/narcotic Violations,,None,Incident,300-BLK S 8TH ST,399 S 8th St,05/25/2022 00:33,"399 S 8th St, Wilmington, NC 28401","(399, South 8th Street, Lake Forest, Wilmingto...","(34.231211, -77.938687, 0.0)",34.231211,-77.938687,0.0
1,Financial Transaction Card Fraud,,None,Incident,700-BLK ANN ST,702 Ann St,05/26/2022 22:00,"702 Ann St, Wilmington, NC 28401","(702, Ann Street, Wilmington, New Hanover Coun...","(34.232171, -77.93996966666667, 0.0)",34.232171,-77.939970,0.0
2,Missing Persons,,None,Incident,400-BLK S 5TH AV,200 N 12TH ST,05/28/2022 12:00,"200 N 12TH ST, Wilmington, NC 28401","(200, North 12th Street, Wilmington, New Hanov...","(34.238867, -77.934319, 0.0)",34.238867,-77.934319,0.0
3,Resisting Arrest/hinder & Delay,,None,Incident,700-BLK NUN ST,700 Nun St,05/30/2022 22:58,"700 Nun St, Wilmington, NC 28401","(700, Nun Street, Wilmington, New Hanover Coun...","(34.230901, -77.939924, 0.0)",34.230901,-77.939924,0.0
4,Mv Theft,,None,Incident,500-BLK DOCK ST,500 Dock St,06/01/2022 15:00,"500 Dock St, Wilmington, NC 28401","(500, Dock Street, Wilmington, New Hanover Cou...","(34.234573, -77.942995, 0.0)",34.234573,-77.942995,0.0
5,B&e Auto Larceny From Auto,,None,Incident,300-BLK S 6TH ST,307 S 6th St,06/02/2022 11:58,"307 S 6th St, Wilmington, NC 28401","(307, South 6th Street, Lake Forest, Wilmingto...","(34.23194679591837, -77.94141046938775, 0.0)",34.231947,-77.941410,0.0
6,Miscellaneous,,None,Incident,600-BLK DOCK ST,608 Dock St,06/05/2022 11:14,"608 Dock St, Wilmington, NC 28401","(608, Dock Street, Wilmington, New Hanover Cou...","(34.234685285714285, -77.94160897959183, 0.0)",34.234685,-77.941609,0.0
7,Mv Theft,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00,"510 Dock St, Wilmington, NC 28401","(510, Dock Street, Wilmington, New Hanover Cou...","(34.234581469387756, -77.94289214285713, 0.0)",34.234581,-77.942892,0.0
8,Possession Of Stolen Vehicles,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00,"510 Dock St, Wilmington, NC 28401","(510, Dock Street, Wilmington, New Hanover Cou...","(34.234581469387756, -77.94289214285713, 0.0)",34.234581,-77.942892,0.0
9,B&e Felony,,None,Incident,400-BLK S 7TH ST,411 S 7th St,07/01/2022 22:00,"411 S 7th St, Wilmington, NC 28401","(411, South 7th Street, Lake Forest, Wilmingto...","(34.23075312244898, -77.93995524489796, 0.0)",34.230753,-77.939955,0.0


In [21]:
df['address'].unique()

array(['399 S 8th St', '702 Ann St', '200 N 12TH ST', '700 Nun St',
       '500 Dock St', '307 S 6th St', '608 Dock St', '510 Dock St',
       '411 S 7th St', '332 S 8th St', '316 Ann St', '224 S 4th St',
       '299 S 6th St', '117 S 7th St', '712 Dock St', '512 Nun St',
       '212 S 7th St', '705 Orange St', '715 Ann St', '516 Dock St',
       '700 Ann St', '200 S 7th St', '410 S 7th St', '711 Nun St'],
      dtype=object)

In [22]:
'''Standardize addresses'''
df['address']=df['address'].str.extract(r'(\d+ .* St).*')
df

,crime,injury,relationship,report_type,block,address,date,full_address,location,point,latitude,longitude,altitude
0,Drug/narcotic Violations,,None,Incident,300-BLK S 8TH ST,399 S 8th St,05/25/2022 00:33,"399 S 8th St, Wilmington, NC 28401","(399, South 8th Street, Lake Forest, Wilmingto...","(34.231211, -77.938687, 0.0)",34.231211,-77.938687,0.0
1,Financial Transaction Card Fraud,,None,Incident,700-BLK ANN ST,702 Ann St,05/26/2022 22:00,"702 Ann St, Wilmington, NC 28401","(702, Ann Street, Wilmington, New Hanover Coun...","(34.232171, -77.93996966666667, 0.0)",34.232171,-77.939970,0.0
2,Missing Persons,,None,Incident,400-BLK S 5TH AV,NaN,05/28/2022 12:00,"200 N 12TH ST, Wilmington, NC 28401","(200, North 12th Street, Wilmington, New Hanov...","(34.238867, -77.934319, 0.0)",34.238867,-77.934319,0.0
3,Resisting Arrest/hinder & Delay,,None,Incident,700-BLK NUN ST,700 Nun St,05/30/2022 22:58,"700 Nun St, Wilmington, NC 28401","(700, Nun Street, Wilmington, New Hanover Coun...","(34.230901, -77.939924, 0.0)",34.230901,-77.939924,0.0
4,Mv Theft,,None,Incident,500-BLK DOCK ST,500 Dock St,06/01/2022 15:00,"500 Dock St, Wilmington, NC 28401","(500, Dock Street, Wilmington, New Hanover Cou...","(34.234573, -77.942995, 0.0)",34.234573,-77.942995,0.0
5,B&e Auto Larceny From Auto,,None,Incident,300-BLK S 6TH ST,307 S 6th St,06/02/2022 11:58,"307 S 6th St, Wilmington, NC 28401","(307, South 6th Street, Lake Forest, Wilmingto...","(34.23194679591837, -77.94141046938775, 0.0)",34.231947,-77.941410,0.0
6,Miscellaneous,,None,Incident,600-BLK DOCK ST,608 Dock St,06/05/2022 11:14,"608 Dock St, Wilmington, NC 28401","(608, Dock Street, Wilmington, New Hanover Cou...","(34.234685285714285, -77.94160897959183, 0.0)",34.234685,-77.941609,0.0
7,Mv Theft,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00,"510 Dock St, Wilmington, NC 28401","(510, Dock Street, Wilmington, New Hanover Cou...","(34.234581469387756, -77.94289214285713, 0.0)",34.234581,-77.942892,0.0
8,Possession Of Stolen Vehicles,,None,Incident,500-BLK DOCK ST,510 Dock St,06/07/2022 22:00,"510 Dock St, Wilmington, NC 28401","(510, Dock Street, Wilmington, New Hanover Cou...","(34.234581469387756, -77.94289214285713, 0.0)",34.234581,-77.942892,0.0
9,B&e Felony,,None,Incident,400-BLK S 7TH ST,411 S 7th St,07/01/2022 22:00,"411 S 7th St, Wilmington, NC 28401","(411, South 7th Street, Lake Forest, Wilmingto...","(34.23075312244898, -77.93995524489796, 0.0)",34.230753,-77.939955,0.0


In [23]:
number, street = df['block'].str.extract(r'(\d+)-BLK(.*)')[0], df['block'].str.extract(r'(\d+)-BLK(.*)')[1]
number

0     300
1     700
2     400
3     700
4     500
5     300
6     600
7     500
8     500
9     400
10    300
11    300
12    300
13    100
14    200
15    200
16    100
17    700
18    500
19    200
20    700
21    700
22    500
23    700
24    200
25    400
26    700
27    700
Name: 0, dtype: object

In [24]:
browser.close()

NameError: name 'browser' is not defined

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   crime         28 non-null     object 
 1   injury        28 non-null     object 
 2   relationship  28 non-null     object 
 3   report_type   28 non-null     object 
 4   block         28 non-null     object 
 5   address       25 non-null     object 
 6   date          28 non-null     object 
 7   full_address  28 non-null     object 
 8   location      28 non-null     object 
 9   point         28 non-null     object 
 10  latitude      28 non-null     float64
 11  longitude     28 non-null     float64
 12  altitude      28 non-null     float64
dtypes: float64(3), object(10)
memory usage: 3.0+ KB


In [34]:
df.groupby('crime')['injury'].count()

crime
B&e Auto Larceny From Auto          1
B&e Felony                          1
Death Investigation                 2
Drug/narcotic Violations            3
Dwi - Alcohol                       2
Financial Transaction Card Fraud    1
Fire Of Unk Origin                  1
Hit & Run-misd Fail To Stop         1
Larceny - All Other                 5
Larceny - From Auto                 3
Miscellaneous                       1
Missing Persons                     1
Mv Theft                            2
Overdose                            1
Possession Of  Stolen Vehicles      1
Resisting Arrest/hinder & Delay     1
Trespass Of Real Property           1
Name: injury, dtype: int64